<a href="https://colab.research.google.com/github/NarreddulaChandrika/chandrika/blob/main/Copy_of_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

eee

In [ ]:
import collections
import pandas as pd
import helper
import numpy as np
import matplotlib.pyplot as plt

from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

In [ ]:
english_sentences = pd.read_csv('/content/drive/MyDrive/small_vocab_en.txt', sep = '\t', names = ['english'])
french_sentences = pd.read_csv('/content/drive/MyDrive/small_vocab_fr.txt', sep = '\t', names = ['french'])

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
english_sentences=english_sentences['english'].tolist()
french_sentences=french_sentences['french'].tolist()

In [ ]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


In [ ]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

In [ ]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # TODO: Implement
    x_tk = Tokenizer()
    x_tk.fit_on_texts(x)

    return x_tk.texts_to_sequences(x), x_tk

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


In [ ]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    # TODO: Implement
    length = max(map(len,x)) if length is None else length

    return pad_sequences(x, maxlen=length, padding='post')

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


In [ ]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)

max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344


In [ ]:
def final_predictions(text):
  y_id_to_word = {value: key for key, value in french_tokenizer.word_index.items()}
  y_id_to_word[0] = '<PAD>'

  sentence = [english_tokenizer.word_index[word] for word in text.split()]
  sentence = pad_sequences([sentence], maxlen=preproc_french_sentences.shape[-2], padding='post')

  return logits_to_text(model1.predict(sentence[:1])[0], french_tokenizer)
def logits_to_text(logits, tokenizer):
  index_to_words = {id: word for word, id in tokenizer.word_index.items()}
  index_to_words[0] = '<PAD>'

  #So basically we are predicting output for a given word and then selecting best answer
  #Then selecting that label we reverse-enumerate the word from id
  r= ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])
  return r

In [ ]:
from keras.models import load_model
model1 = load_model('/content/drive/MyDrive/model.h5')

In [ ]:
import re
def ret(txt):
  try:
    k=final_predictions(re.sub(r'[^\w]', ' ', txt))
    k=k.replace('<PAD>','')
  except:
    k='invalid sentence give another one'
  return k

eee

eee

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import string
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [ ]:
!unzip '/content/drive/MyDrive/speech.zip'

Archive:  /content/drive/MyDrive/speech.zip
  inflating: ArticlesApril2017.csv   
  inflating: ArticlesApril2018.csv   
  inflating: ArticlesFeb2017.csv     
  inflating: ArticlesFeb2018.csv     
  inflating: ArticlesJan2017.csv     
  inflating: ArticlesJan2018.csv     
  inflating: ArticlesMarch2017.csv   
  inflating: ArticlesMarch2018.csv   
  inflating: ArticlesMay2017.csv     
  inflating: CommentsApril2017.csv   
  inflating: CommentsApril2018.csv   
  inflating: CommentsFeb2017.csv     
  inflating: CommentsFeb2018.csv     
  inflating: CommentsJan2017.csv     
  inflating: CommentsJan2018.csv     
  inflating: CommentsMarch2017.csv   
  inflating: CommentsMarch2018.csv   
  inflating: CommentsMay2017.csv     


In [ ]:
curr_dir = '/content/'
all_headlines = []
for filename in os.listdir(curr_dir):
    if 'Articles' in filename:
        article_df = pd.read_csv(curr_dir + filename)
        all_headlines.extend(list(article_df.headline.values))
        break
all_headlines = [line for line in all_headlines if line!= "Unknown"]
all_headlines[:5]

[' G.O.P. Leadership Poised to Topple Obama’s Pillars',
 'Fractured World Tested the Hope of a Young President',
 'Little Troublemakers',
 'Angela Merkel, Russia’s Next Target',
 'Boots for a Stranger on a Bus']

In [ ]:
def clean_s1(txt):
    txt = "".join(t for t in txt if t not in string.punctuation).lower()
    txt = txt.encode("utf8").decode("ascii",'ignore')
    return txt
corpus = [clean_s1(x) for x in all_headlines]
corpus[:5]

[' gop leadership poised to topple obamas pillars',
 'fractured world tested the hope of a young president',
 'little troublemakers',
 'angela merkel russias next target',
 'boots for a stranger on a bus']

In [ ]:
tokenizer = Tokenizer()
def get_sequence_of_tokens(corpus):
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1

    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words

input_sequences, total_words = get_sequence_of_tokens(corpus)
input_sequences[:8]

[[73, 313],
 [73, 313, 616],
 [73, 313, 616, 3],
 [73, 313, 616, 3, 617],
 [73, 313, 616, 3, 617, 205],
 [73, 313, 616, 3, 617, 205, 314],
 [618, 38],
 [618, 38, 619]]

In [ ]:
for j in range(8):
    s = input_sequences[j]
    for i in range(1, len(s)+1):
        print(tokenizer.index_word[s[i-1]], end = ' ')
    print()

gop leadership 
gop leadership poised 
gop leadership poised to 
gop leadership poised to topple 
gop leadership poised to topple obamas 
gop leadership poised to topple obamas pillars 
fractured world 
fractured world tested 


In [ ]:
def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = tf.keras.utils.to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len

predictors, label, max_sequence = generate_padded_sequences(input_sequences)
max_sequence = max_sequence-1
print("Length of Max Sequence:",max_sequence)
for i in range(3):
    print(predictors[i], list(label[i]).index(1))

Length of Max Sequence: 20
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 73] 313
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
  73 313] 616
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  73
 313 616] 3


In [ ]:
total_words

2217

eee

In [ ]:
model = Sequential()
model.add(Embedding(total_words,20,input_length=max_sequence))
model.add(Bidirectional(LSTM(units=100, return_sequences=True)))
model.add(LSTM(units=100))
model.add(Dropout(0.5))
model.add(Dense(units=total_words, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 20)            44340     
                                                                 
 bidirectional (Bidirection  (None, 20, 200)           96800     
 al)                                                             
                                                                 
 lstm_1 (LSTM)               (None, 100)               120400    
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense (Dense)               (None, 2217)              223917    
                                                                 
Total params: 485457 (1.85 MB)
Trainable params: 485457 (1.85 MB)
Non-trainable params: 0 (0.00 Byte)
____________________

In [ ]:
model.fit(predictors,label,epochs=100,verbose=5)

Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
Epoch 46/100
Epoch 47/100
Epoch 48/100
Epoch 49/100
Epoch 50/100
Epoch 51/100
Epoch 52/100
Epoch 53/100
Epoch 54/100
Epoch 55/100
Epoch 56/100
Epoch 57/100
Epoch 58/100
Epoch 59/100
Epoch 60/100
Epoch 61/100
Epoch 62/100
Epoch 63/100
Epoch 64/100
Epoch 65/100
Epoch 66/100
Epoch 67/100
Epoch 68/100
Epoch 69/100
Epoch 70/100
Epoch 71/100
Epoch 72/100
Epoch 73/100
Epoch 74/100
Epoch 75/100
Epoch 76/100
Epoch 77/100
Epoch 78

In [ ]:
from tensorflow.keras.models import save_model
save_model(model, "prompt.h5")

<ipython-input-23-4e625edfd19d>:2: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  save_model(model, "prompt.h5")


eee

In [ ]:
from tensorflow.keras.models import load_model

# load model
model2 = load_model('/content/prompt.h5')

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import string
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [ ]:
def make_pred(sentence, limit):
    for i in range(limit):
        tokenized_words = tokenizer.texts_to_sequences([sentence])[0]
        tokenized_words = pad_sequences([tokenized_words], maxlen=max_sequence, padding='pre')
        pred_word = np.argmax(model2.predict(tokenized_words))
        pred = tokenizer.index_word[pred_word]
        sentence += " " + pred
    return sentence

sentence = "I know how to"
length_words = 2
output_pred = make_pred(sentence,length_words)
print(output_pred)

1/1 [==============================] - 0s 57ms/step
I know how to partisan partisan


In [ ]:
!pip install -q https://github.com/tugstugi/dl-colab-notebooks/archive/colab_utils.zip
!pip install speechRecognition
!pip install gTTS
!pip install google-colab

!pip install wikipedia
!pip install wolframalpha

     - 3.0 kB ? 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=0967d48e6995460da20308fa2f527310e84874d2a0032bb811abc4f6f323a2a5
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [ ]:

from IPython.display import Audio, display, clear_output, Javascript
import ipywidgets as widgets
import numpy as np
from scipy.io import wavfile
from dl_colab_notebooks.audio import record_audio, upload_audio
import speech_recognition as sr



import random #It is used to obtain a random value from a group of values
import wikipedia #Used to get articles from wikipedia and summarize them if necessary
import wolframalpha #Used in the core of the VA
import datetime #Used to get the current time
import os #For system related functions
import sys #For system related functions
from gtts import gTTS
import librosa
import time

In [ ]:

SAMPLE_RATE = 16000
record_seconds =   3#@param {type:"number", min:1, max:10, step:1}
text_recognized = "default_text"

In [ ]:
def speak(to_speak="Default"):
  tts = gTTS(to_speak) #Provide the string to convert to speech
  tts.save('out.wav') #save the string converted to speech as a .wav file
  sound_file = 'out.wav'
  display(Audio(sound_file, autoplay=True))
  time.sleep(librosa.get_duration(filename='out.wav'))
  return 200

def _recognize(audio):
  # display(Audio(audio, rate=SAMPLE_RATE, autoplay=True))
  wavfile.write('in.wav', SAMPLE_RATE, (32767*audio).astype(np.int16))
  print('\n')
  filename = "in.wav"
  r = sr.Recognizer()
  with sr.AudioFile(filename) as source:
      # listen for the data (load audio to memory)
      try:
        audio_data = r.record(source)
        # recognize (convert from speech to text)
        text = r.recognize_google(audio_data)
        return text
      except Exception as e:
        print(e)
        print("Could you speak more clearly")
        return "None"


def _record_audio():
  clear_output()
  audio = record_audio(record_seconds, sample_rate=SAMPLE_RATE)
  text_recognized = _recognize(audio)
  print(f"Users says: {text_recognized}\n")
  return text_recognized

def takeCommand():
  button = widgets.Button(description="Record Speech")
  return _record_audio()
  # button.on_click(_record_audio)
  # display(button)

def open_web(url):
    display(Javascript('window.open("{url}");'.format(url=url)))

def wishMe():
    hour = int(datetime.datetime.now().hour)
    if hour>= 0 and hour<12:
        speak("Good Morning Sir !")
    elif hour>= 12 and hour<18:
        speak("Good Afternoon Sir !")
    else:
        speak("Good Evening Sir !")
    name =("translator")
    speak("I am your Assistant")
    speak(name)

def usrname():
    speak("What should i call you sir")
    uname = takeCommand()
    speak("Welcome")
    speak(uname)

In [ ]:
import time
def bot(query=""):
    k=True
    while k==True:
        speak("How can I help you")
        time.sleep(1)
        query = takeCommand().lower()
        print(query)
        if 'wikipedia' in query:
            speak('Searching Wikipedia...')
            query = query.replace("wikipedia", "")
            results = wikipedia.summary(query, sentences = 3)
            speak("Here is what wikipedia says")
            print(results)
            speak(results)
            k=True

        elif 'open youtube' in query:
            speak("Yes opening")
            open_web('https://www.youtube.com');
            k=True

        elif 'open forum' in query:
            speak("Yes opening " + str("forum.aistudent.community"))
            open_web("https://www.forum.aistudent.community")
            k=True

        elif 'open google' in query:
            speak("Yes opening " + str("www.google.co.in"))
            open_web("https://www.google.co.in")
            k=True

        elif 'open stack overflow' in query:
            speak("Yes opening " + str("stackoverflow.com"))
            open_web("https://www.stackoverflow.com")
            k=True

        elif 'how are you' in query:
            speak("I am fine, Thank you")
            speak("How are you, opening " + str())
            k=True

        elif 'fine' in query or "good" in query:
            speak("It's good to know that your fine")
            k=True

        elif "change name" in query:
            speak("What would you like to call me, Sir ")
            name = takeCommand()
            speak("Thanks for naming me")
            k=True

        elif "what's your name" in query or "What is your name" in query:
            speak("My friends call me")
            name =("Arnold")
            speak(name)
            print("My friends call me", name)
            k=True

        elif 'search' in query or 'play' in query:
            query = query.replace("search", "")
            query = query.replace("play", "")
            open_web(query)
            k=True

        elif "who are you" in query:
            speak("I am your virtual assistant")
            k=True

        elif "where is" in query:
            query = query.replace("where is", "")
            location = query
            speak("User asked to Locate")
            speak(location)
            open_web("https://www.google.com/maps/search/?api=1&" + location + "")
            k=True

        elif "will you be my friend" in query:
            speak("Sure, why not")
            k=True
        elif "generate text" in query:
            speak("give a prompt")
            prompt = takeCommand().lower()
            speak("set the limit")
            length = takeCommand().lower()
            if length == 'none':
              speak(make_pred(prompt, 10))
            else:
              speak(make_pred(prompt, int(length)))
            k=True
            time.sleep(3)
        elif "translate" in query:
          t=True
          while t:
            speak("give me a sentence to translate")
            txt=takeCommand().lower()
            p=ret(txt)
            if "invalid" in p:
              speak(p)
              t=True
            else:
              speak(p)
              t=False
          time.sleep(2)


        elif 'exit' in query:
            speak("Thanks for giving me your time")
            k=False
            exit()

In [ ]:
bot()

Starting recording for 3 seconds...


<IPython.core.display.Javascript object>

Finished recording!


Users says: exit

exit


<ipython-input-30-ce6ee7c19223>:6: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  time.sleep(librosa.get_duration(filename='out.wav'))


eee final

In [ ]:
!unzip '/content/drive/MyDrive/en-fr.csv.zip'

unzip:  cannot find or open /content/drive/MyDrive/en-fr.csv.zip, /content/drive/MyDrive/en-fr.csv.zip.zip or /content/drive/MyDrive/en-fr.csv.zip.ZIP.


In [ ]:
#import all libraries
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional,LSTM, Dropout

from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
from keras.callbacks import ModelCheckpoint

In [ ]:
df  =pd.read_csv('/content/en-fr.csv',nrows=20000)
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '/content/en-fr.csv'

In [ ]:
eng = df['en']
fr = df['fr']

In [ ]:
import collections
english_words_counter = collections.Counter([word for sentence in eng for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in fr for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in eng for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in fr for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

NameError: name 'collections' is not defined

In [ ]:
def tokenize(x):

    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(x)
    return tokenizer.texts_to_sequences(x), tokenizer

In [ ]:
def pad(x, length=None):
    if length is None:
        length = max([len(sentence) for sentence in x])
    return pad_sequences(x, maxlen = 55, padding = 'post')

In [ ]:
def preprocess(x, y):

    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

In [ ]:
preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer = preprocess(eng, fr)

In [ ]:
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

In [ ]:
def logits_to_text(logits, tokenizer):

    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

In [ ]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):

    learning_rate = 0.003

    # Build the layers
    model = Sequential()
    model.add(Embedding(french_vocab_size, 256, input_length=input_shape[1], input_shape=input_shape[1:]))
    model.add(Bidirectional(GRU(256, return_sequences=True)))
    model.add(TimeDistributed(Dense(1024, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(english_vocab_size, activation='softmax')))

    # Compile model
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

In [ ]:
tmp_x.shape
preproc_english_sentences.shape

In [ ]:
# Reshape the input
tmp_x = pad(preproc_french_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

# Train
model = bd_model(
    tmp_x.shape,
    preproc_english_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)

model.summary()

model.fit(tmp_x, preproc_english_sentences, batch_size=64, epochs=5, validation_split=0.2)